## Ciencia de Datos Aplicada - Curso Capstone Proyecto Final

### Agrupación y Segmentación de Vecindarios en la Ciudad de Toronto, Canadá

Use el Cuaderno para crear el código para rastrear la siguiente página de Wikipedia, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, para obtener los datos que están en la tabla de códigos postales y transformar los datos en un marco de datos de pandas 

#### Se Instala librerias 

In [14]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

### Web Scrapin de los Datos 

Por medio de **BeatifulSoup** se hace *Web Scraping* para obtener los datos de la página web que
contiene los códigos postales de Toronto.

In [15]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")

Una vez hecha la sopa de datos se traen solo los datos de la tabla ya que contienen los codigos
postales de Toronto.

In [16]:
table = soup.find('table')

In [17]:
file=[]
ncol=list(range(0,9))

for i in ncol :
    
    for row in table.find_all('tr'): # in html table row is represented by the table
    # Get all columns in each row.
        cols = row.find_all('p') # in html a column is represented by the tag td or p
        file.append(cols[i].getText())

    # convert to dataframe:
df= pd.DataFrame(file)

#### Tabla de Códigos Postales

Despues de traer los datos que estan contenidos en la tabla, por medio de **pandas** se le da formato a la tabla para obtener la salida deseada.

In [24]:
df["Postal Code"]=df[0].str[0:3]
df["Borough"]=df[0].str[3:].str.split("(",n=1,expand=True)[0]
df["Neigh"]=df[0].str.rsplit("(",n=2,expand=True)[1].str.split(")",n=1,expand=True)
df["Neighbourhood"]=df["Neigh"].str.replace(" / ",",")

In [25]:
postalcode_list=df[(df["Borough"]!="Not assigned\n")]
postalcode_list=postalcode_list.drop([0],axis=1)

In [26]:
postalcode_list.head()

,Postal Code,Borough,Neigh,Neighbourhood
1,M1B,Scarborough,Malvern / Rouge,"Malvern,Rouge"
2,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,"Rouge Hill,Port Union,Highland Creek"
3,M1E,Scarborough,Guildwood / Morningside / West Hill,"Guildwood,Morningside,West Hill"
4,M1G,Scarborough,Woburn,Woburn
5,M1H,Scarborough,Cedarbrae,Cedarbrae


In [27]:
postalcode_list.shape

(103, 4)

### Cordenadas de Latitud y Longitud

Por medio del paquete Geocoder Python: https://geocoder.readthedocs.io/index.html. Se obtienen los datos de Latitud y Longitud para cada código postal.

Para asegurarse de obtener las coordenadas de todos los vecindarios, se puede ejecutar un ciclo while para cada código postal. 

In [28]:
#"http://cocl.us/Geospatial_data"
Geospatial_Coordinates=pd.read_csv("Geospatial_Coordinates.csv")

In [29]:
coord_list=pd.merge(postalcode_list,Geospatial_Coordinates,on="Postal Code")

In [30]:
coord_list.head()

,Postal Code,Borough,Neigh,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,"Rouge Hill,Port Union,Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,Cedarbrae,43.773136,-79.239476
